In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

import numpy as np
from sklearn.metrics import confusion_matrix

import os
import pandas as pd
import time
import warnings
from tqdm.notebook import tqdm

import librosa
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(f"Using device: {device}")

print(torch.cuda.device_count())
print(torch.cuda.get_device_name())

warnings.filterwarnings('ignore')

Using device: cuda
2
Quadro GP100


In [6]:
class FeatureDataset(Dataset):
    def __init__(self, csv_file, transform=None, target_transform=None):
        self.df = pd.read_csv(csv_file)
        self.transform = transform
        self.target_transform = target_transform
        self.label_map = {
            'Drone': 0,
            'No Drone': 1
        }

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        features = torch.FloatTensor(self.df.iloc[idx, 2:])
        label = self.df.iloc[idx, 1]
        
        if self.target_transform:
            label = self.target_transform(label)
        
        return torch.tensor(features), torch.tensor(label)
    

# Usage example
csv_file = "/home/distasiom/Documents/Summer2024/data/complete_dataset_features.csv"

# Define any transformations if needed
transform = None
target_transform = None

dataset = FeatureDataset(csv_file, transform=transform, target_transform=target_transform)

In [7]:
train_size = int(.8 * len(dataset))
test_size = int(.75 * (len(dataset) - train_size))
val_size = len(dataset) - train_size - test_size

train_dataset, test_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, test_size, val_size])
print(train_dataset)
print(train_dataset[0])

train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True
    )

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False
)

val_loader = DataLoader(
    val_dataset,
    batch_size=64,
    shuffle=False
)

(tensor([-141.4258, -106.4519, -123.4676,  ...,    0.2202,    0.2202,
           0.2202]), tensor(1))


In [ ]:
for feature, label in train_loader:
    print(label)

In [ ]:
print(train_dataset[7][0].shape)


In [ ]:
# Run 1 small model (36,606,465 parameters)
class LNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq = nn.Sequential(
            nn.LazyLinear(4096),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyLinear(4096),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyLinear(1024),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyLinear(1024),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyLinear(512),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyLinear(256),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyLinear(1)
        )

    def forward(self, x):
        logits = self.seq(x).squeeze(dim=1)
        return logits


In [8]:
# Run 2 larger model (47,104,513 parameters)
class LNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq = nn.Sequential(
            nn.LazyLinear(4096),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyLinear(4096),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyLinear(2048),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyLinear(2048),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyLinear(1024),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyLinear(1024),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyLinear(512),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyLinear(256),
            nn.LazyBatchNorm1d(),
            nn.ReLU(),
            nn.LazyLinear(1)
        )

    def forward(self, x):
        logits = self.seq(x).squeeze(dim=1)
        return logits


In [9]:
def split_seconds(seconds):
    minutes = seconds // 60
    hours = minutes // 60
    days = hours // 24
    return seconds % 60, minutes % 60, hours % 24, days


In [10]:
def main():
    # Define model
    model = LNN()

    # Cuda setup
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # device = torch.device("cpu")
    model = model.to(device)
   
    print(f"Using device: {device}")
    # Optimizer setup
    optimizer = Adam(model.parameters(), lr=1e-4)
   
    # Loss function
    loss_fn = nn.BCEWithLogitsLoss(reduction="mean")
   
    # Number of epochs
    num_epochs = 64
   
    # Train or load model?
    print("Training model....")
    start = time.time()
    for epoch in tqdm(range(num_epochs), desc="Epochs"):
        
        model.train()
        for batch_idx, (features, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)):
            features, labels = features.to(device), labels.to(device)
            optimizer.zero_grad()
           
            # CNN forward pass
            logits = model(features)
            loss = loss_fn(logits, labels.to(float))
            loss.backward()
            optimizer.step()

        model.eval()
        num_test = 0
        num_correct = 0
        total_loss = 0
        with torch.no_grad():
            for features, labels in tqdm(test_loader, desc="Testing"):
                features, labels = features.to(device), labels.to(device)
                logits = model(features)
                loss = loss_fn(logits, labels.to(float))
                total_loss += loss.item()
                preds = torch.round(torch.sigmoid(logits))
                num_test += labels.size(0)
                num_correct += preds.eq(labels).sum().item()
    
        tqdm.write(f"Epoch {epoch+1}/{num_epochs}, Test accuracy: {num_correct / num_test * 100:.2f}, Total loss: {total_loss}")

#         torch.save(model.state_dict(), f"/home/distasiom/Documents/Summer2024/models/dnd_mfcc_classifier/epoch_{str(epoch)}.pt")
   
    end = time.time()
    seconds, minutes, hours, days = split_seconds(end - start)
    print(f"Training Runtime: {int(days)}d {int(hours)}h {int(minutes)}m {seconds:.2f}s")
   
    # Evaluate model on validation data
    model.eval()
    print("Evaluating model....")
    start = time.time()
    total_loss = 0
    num_test = 0
    num_correct = 0
   
    with torch.no_grad():
        for features, labels in tqdm(val_loader, desc="Validating"):
            features, labels = features.to(device), labels.to(device)
            logits = model(features)
            loss = loss_fn(logits, labels.to(float))
            total_loss += loss.item()
            preds = torch.round(torch.sigmoid(logits))
            num_test += labels.size(0)
            num_correct += preds.eq(labels).sum().item()
   
    print(f"Test accuracy: {num_correct / num_test * 100:.2f}, Total loss: {total_loss}")
    end = time.time()
    seconds, minutes, hours, days = split_seconds(end - start)
    print(f"Testing Runtime: {int(days)}d {int(hours)}h {int(minutes)}m {seconds:.2f}s")

In [ ]:
main()

In [11]:
# Print number of parameters
model = LNN()
state = torch.load("/home/distasiom/Documents/Summer2024/models/dnd_mfcc_classifier_large/epoch_0.pt")
model.load_state_dict(state)
total_params = sum([torch.numel(p) for p in model.parameters()])
print(f"Total parameters: {total_params}")

Total parameters: 47104513


In [12]:
# Define model
model = LNN()
# Cuda setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
model = model.to(device)

print(f"Using device: {device}")

directory = "/home/distasiom/Documents/Summer2024/models/dnd_mfcc_classifier_large"
acc_epoch = None
max_acc = 0
loss_epoch = None
min_loss = float('inf')

for file in os.listdir(directory):
    # Optimizer setup
    optimizer = Adam(model.parameters(), lr=1e-3)

    # Loss function
    loss_fn = nn.BCEWithLogitsLoss(reduction="mean")

    # Load model
    filepath = os.path.join(directory, file)
    state = torch.load(filepath)
    model.load_state_dict(state)

    # Evaluate model on validation data
    model.eval()
    print(f"Evaluating model {file}")
    total_loss = 0
    num_test = 0
    num_correct = 0

    with torch.no_grad():
        for features, labels in tqdm(val_loader, desc="Validating"):
            features, labels = features.to(device), labels.to(device)
            logits = model(features)
            loss = loss_fn(logits, labels.to(float))
            total_loss += loss.item()
            preds = torch.round(torch.sigmoid(logits))
            num_test += labels.size(0)
            num_correct += preds.eq(labels).sum().item()

    accuracy = num_correct / num_test * 100
    if accuracy > max_acc:
        max_acc = accuracy
        acc_epoch = file
    if total_loss < min_loss:
        min_loss = total_loss
        loss_epoch = file

    print(f"Test accuracy: {accuracy:.2f}, Total loss: {total_loss}")

print(f"Best accuracy: {max_acc} at {acc_epoch}")
print(f"Best loss: {min_loss} at {loss_epoch}")

Using device: cuda
Evaluating model epoch_0.pt


Validating:   0%|          | 0/55 [00:00<?, ?it/s]

Test accuracy: 89.52, Total loss: 12.015218783603684
Evaluating model epoch_1.pt


Validating:   0%|          | 0/55 [00:00<?, ?it/s]

Test accuracy: 90.21, Total loss: 10.812226473866227
Evaluating model epoch_2.pt


...

Test accuracy: 94.66, Total loss: 6.893292412047079
Evaluating model epoch_62.pt


Validating:   0%|          | 0/55 [00:00<?, ?it/s]

Test accuracy: 94.60, Total loss: 6.611815895889569
Evaluating model epoch_63.pt


Validating:   0%|          | 0/55 [00:00<?, ?it/s]

Test accuracy: 94.92, Total loss: 6.890223489045573
Best accuracy: 95.06208489748774 at epoch_54.pt
Best loss: 6.150015494908922 at epoch_54.pt


# Small

Best accuracy: 90.70170372509385 at epoch_14.pt

Best loss: 10.261200130774341 at epoch_7.pt

# Large

Best accuracy: 95.06208489748774 at epoch_54.pt

Best loss: 6.150015494908922 at epoch_54.pt